In [13]:
import pandas as pd
import numpy as np
import altair as alt
import re
import statsmodels as sm
from statsmodels.tsa.vector_ar.var_model import VAR
from sklearn.feature_selection import VarianceThreshold

In [2]:
data = pd.read_csv('./data/TrainingSet.csv', index_col=0)
submission_rows = pd.read_csv('./data/SubmissionRows.csv', index_col=0)

In [3]:
def clean_up_year_columns(columns):
  return list(map(lambda x: re.sub(r' \[YR(.*)', '', x), columns))

In [4]:
data.columns = clean_up_year_columns(data.columns) 

In [5]:
data.head(20)

,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,...,2001,2002,2003,2004,2005,2006,2007,Country Name,Series Code,Series Name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.769214e+00,Afghanistan,allsi.bi_q1,(%) Benefits held by 1st 20% population - All ...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.027746e+00,Afghanistan,allsp.bi_q1,(%) Benefits held by 1st 20% population - All ...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.244887e+00,Afghanistan,allsa.bi_q1,(%) Benefits held by 1st 20% population - All ...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.293310e+01,Afghanistan,allsi.gen_pop,(%) Generosity of All Social Insurance
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.899681e+01,Afghanistan,allsp.gen_pop,(%) Generosity of All Social Protection
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.242306e+01,Afghanistan,allsa.gen_pop,(%) Generosity of All Social Safety Nets
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.205600e-01,Afghanistan,allsi.cov_pop,(%) Program participation - All Social Insurance
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.496313e+01,Afghanistan,allsp.cov_pop,(%) Program participation - All Social Protection
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.451376e+01,Afghanistan,allsa.cov_pop,(%) Program participation - All Social Safety ...
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,Afghanistan,lm_ub.cov_pop,(%) Program participation - Unemp benefits and...


In [6]:
included_years = list(map(lambda x: str(x), range(1972, 2008)))

In [7]:
data.columns

Index(['1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989',
       '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       'Country Name', 'Series Code', 'Series Name'],
      dtype='object')

In [8]:
data.head()

,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,...,2001,2002,2003,2004,2005,2006,2007,Country Name,Series Code,Series Name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.769214,Afghanistan,allsi.bi_q1,(%) Benefits held by 1st 20% population - All ...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.027746,Afghanistan,allsp.bi_q1,(%) Benefits held by 1st 20% population - All ...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.244887,Afghanistan,allsa.bi_q1,(%) Benefits held by 1st 20% population - All ...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.933105,Afghanistan,allsi.gen_pop,(%) Generosity of All Social Insurance
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18.996814,Afghanistan,allsp.gen_pop,(%) Generosity of All Social Protection


In [9]:
pred_data = data.copy()

In [84]:
submission_series = data.loc[submission_rows.index]['Series Name'].unique()
submission_series

array(['Ensure environmental sustainability',
       'Develop a global partnership for development: Internet Use',
       'Reduce child mortality', 'Combat HIV/AIDS',
       'Achieve universal primary education', 'Improve maternal health',
       'Combat malaria and other diseases'], dtype=object)

In [11]:
def get_country_time_series(data, country_name):
    country_data = data[data['Country Name'] == country_name]
    
    time_series = country_data.melt(
        id_vars=['Series Name'], 
        value_vars=included_years, 
        var_name='year', 
        value_name='index'
    ).pivot(index='year', columns='Series Name', values='index')
    
    time_series = time_series.dropna(axis='columns', thresh=4)
    time_series.index = pd.to_datetime(time_series.index)
    time_series = time_series.interpolate(method='slinear', axis=0, limit_direction='both').ffill().bfill()
    
    return time_series

In [96]:
final_pred_df = None

In [97]:
%%capture

for country in data['Country Name'].unique():
    time_series = get_country_time_series(data, country)
    time_series_diffed = time_series.diff().dropna()
    
    selector = VarianceThreshold()
    selector.fit(time_series_diffed)
    selected_columns = time_series_diffed.columns[selector.get_support()]
    time_series_diffed = time_series_diffed[selected_columns]
    
    model = VAR(endog=time_series_diffed)
    model_fit = model.fit()
    five_year_pred = model_fit.forecast(model_fit.endog, steps=5)
    pred_df = pd.DataFrame(five_year_pred)
    pred_df.index = pd.DatetimeIndex(['2008-01-01', '2009-01-01', '2010-01-01', '2011-01-01', '2012-01-01'], name='year')
    pred_df.columns = selected_columns
    
    new_time_series = pd.concat([time_series.iloc[[0]], time_series_diffed, pred_df])
    new_time_series = new_time_series.apply(np.cumsum)
    
    pred_time_series = new_time_series.tail(5).reset_index()
    melted_pred_time_series = pred_time_series.melt(id_vars='year', var_name='Series Name', value_name='Series index')
    proper_time_series = melted_pred_time_series.pivot(index='Series Name', columns='year', values='Series index').reset_index()
    
    proper_time_series.columns = ['Series Name', '2008', '2009', '2010', '2011', '2012']
    proper_time_series['Country Name'] = country
    final_pred_df = pd.concat([final_pred_df, proper_time_series], ignore_index=True)

In [98]:
final = pd.merge(
    data,
    final_pred_df, 
    left_on=['Country Name', 'Series Name'], 
    right_on=['Country Name', 'Series Name'], 
    how='outer'
)

In [99]:
final.index = data.index

In [126]:
submission_df = final.loc[submission_rows.index].fillna(value=final['2007'])

In [133]:
submission_df['2008'] = submission_df['2008'].fillna(submission_df['2007'])
submission_df['2012'] = submission_df['2012'].fillna(submission_df['2007'])

In [137]:
final_submission = submission_df[['2008', '2012']]

In [140]:
submission_rows.shape

(737, 2)

In [141]:
final_submission.columns = submission_rows.columns

In [143]:
final_submission.to_csv('sub.csv')